<h1>Seminar 1: Optimizing Conference Paper </h1>
Assignment Using Genetic Algorithms <div>
Intelligent Systems <div>
October 28,

In [2]:
import numpy as np
%pip install pygad
import pygad
import random
import json
import itertools

Note: you may need to restart the kernel to use updated packages.


Data

In [3]:

with open('datasets/easy_dataset_1.json', 'r') as file:
    data = json.load(file)
num_papers = data['num_papers']
num_reviewers = data['num_reviewers']
reviewer_capacity = data['reviewer_capacity']
min_reviews_per_paper = data['min_reviews_per_paper']
max_reviews_per_paper = data['max_reviews_per_paper']
#print(num_papers, num_reviewers, reviewer_capacity, min_reviews_per_paper, max_reviews_per_paper)

num_generations = 1000
population_size = 500
num_parents_mating = 5
P = np.array(data['preferences'])
F = np.array(data['friendships'])
A = np.array(data['authorship'])

<h3> Print functions </h3>

In [ ]:

def print_solution(solution_matrix):
    solution_matrix = solution_matrix.reshape((num_reviewers, num_papers))

    header = "Reviewers\\Papers||| " + " | ".join([f"Paper {i+1}" for i in range(num_papers)])
    print(header)
    print("-" * len(header))

    # Print each reviewer's assignments
    for reviewer_idx in range(num_reviewers):
        row = f"Reviewer {reviewer_idx+1:<6} |||   " + "  |   ".join(f"{value:<4}" for value in solution_matrix[reviewer_idx])
        print(row)


<h2>Task 1 - Representation <h2>

In [ ]:

def create_fitness_function(fitnes_penalty):
    def fitness_func(ga_instance, solution, solution_idx):
        preference_score = np.sum(solution * P.flatten())
        #print(preference_score)

        penalty = 0
        penalty += np.sum(solution * A.flatten()) * fitnes_penalty["authorship"]
        #print(penalty)
        
        reshaped_sol = solution.reshape((num_reviewers, num_papers))
        #print(reshaped_sol)

        #penalty for exceeding reviewer capacity, not meeting min reviews per paper, exceeding max reviews per paper
        per_reviewer = np.sum(reshaped_sol, axis=1)
        per_paper = np.sum(reshaped_sol, axis=0)

        penalty += (np.maximum(0, per_paper - max_reviews_per_paper) *fitnes_penalty["max_reviews"]).sum()
        penalty += (np.maximum(0, min_reviews_per_paper - per_paper) *fitnes_penalty["min_reviews"]).sum() 
        penalty += (np.maximum(0, per_reviewer - reviewer_capacity) * fitnes_penalty["reviewer_capacity"]).sum() 
        
       
        # how many papers have reviewers that are friends
        co_review_matrix = np.dot(reshaped_sol, reshaped_sol.T)
        friend_review_counts = F * co_review_matrix 
        penalty += np.sum(friend_review_counts) // 2 * fitnes_penalty["friends"]
        #print(penalty)

        #how many friends reviewed papers that their friends authored
        authored_papers_by_friends = np.dot(F, A)
        penalty_matrix = reshaped_sol * authored_papers_by_friends
        penalty += np.sum(penalty_matrix) * fitnes_penalty["friends"]
        #print(penalty)
        #return  preference_score - penalty
        return  preference_score - penalty
    return fitness_func